In [8]:
import ee
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [9]:
def dataset_value(dataset, lon, lat):
    point = ee.Geometry.Point([lon, lat])
    if isinstance(dataset, ee.imagecollection.ImageCollection):
        dataset = dataset.mean()
    value = dataset.reduceRegion(
        reducer=ee.Reducer.first(), 
        geometry=point, 
        scale=30
    ).getInfo()
    return value

In [10]:
elevation = ee.Image("NASA/NASADEM_HGT/001").select('elevation')
temperature = ee.ImageCollection("MODIS/061/MOD11A1").select('LST_Day_1km')
biomass = ee.ImageCollection("WCMC/biomass_carbon_density/v1_0").select('carbon_tonnes_per_ha')
fires = ee.FeatureCollection("JRC/GWIS/GlobFire/v2/FinalPerimeters")

In [21]:
n=3
fire_feature = ee.Feature(fires.toList(n).get(n-1))
fire_bounds = fire_feature.geometry().bounds()
buffer_distance = 1

# Create a rectangle around the fire perimeter with a buffer
rectangle = fire_bounds.buffer(buffer_distance).bounds()

In [5]:
def generate_grid(corners, point_spacing):
    coords = []
    min_lon, min_lat = corners[0]
    max_lon, max_lat = corners[1]

    lon = min_lon
    while lon < max_lon:
        lat = min_lat
        while lat < max_lat:
            coords.append([lon, lat])
            lat += point_spacing
        lon += point_spacing
    return coords


In [6]:
# Define the spacing between points (in degrees)
point_spacing = 0.001  # Example: approximately 111 meters at the equator

# Extract the lower-left and upper-right corners of the rectangle
rect_coords = rectangle.bounds().getInfo()['coordinates'][0]
lower_left = rect_coords[0]  # Lower-left corner
upper_right = rect_coords[2]  # Upper-right corner

# Generate the grid
grid_points = generate_grid([lower_left, upper_right], point_spacing)

In [18]:
import numpy as np
data = []
for lon, lat in grid_points:
    elev = dataset_value(elevation, lon, lat)
    temp = dataset_value(temperature, lon, lat)
    bio = dataset_value(biomass, lon, lat)
    burned = fire_feature.geometry().contains(ee.Geometry.Point([lon, lat])).getInfo()
    data.append([elev.get('elevation'), temp.get('LST_Day_1km'), bio.get('carbon_tonnes_per_ha'), burned])

np_data = np.array(data)
print(np_data)

In [20]:

centroid = fire_feature.geometry().centroid()

# Extract the coordinates of the centroid
centroid_coords = centroid.coordinates().getInfo()

# Print the coordinates
print("Fire Centroid Coordinates:", centroid_coords)

Fire Centroid Coordinates: [-35.804761001213365, -9.410416676692531]


In [18]:

fire_id = fire_feature.get('Id').getInfo()

# Print the ID
print("Fire ID:", fire_id)

Fire ID: 9331845
